# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [59]:
# import libraries
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

from pyspark.sql.functions import col

In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [3]:
# Read in full sparkify dataset
event_data = "../input/mini_sparkify_event_data.json"
df = spark.read.json(event_data)

In [4]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

The first five rows of the dataset.

In [5]:
df.take(5)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

Schema information

* artist: Artist name (ex. Daft Punk)
* auth: User authentication status (ex. Logged)
* firstName: User first name (ex. Colin)
* gender: Gender (ex. F or M)
* itemInSession: Item count in a session (ex. 52)
* lastName: User last name (ex. Freeman)
* length: Length of song (ex. 223.60771)
* level: User plan (ex. paid)
* location: User's location (ex. Bakersfield)
* method: HTTP method (ex. PUT)
* page: Page name (ex. NextSong)
* registration: Registration timestamp (unix timestamp) (ex. 1538173362000)
* sessionId: Session ID (ex. 29)
* song: Song (ex. Harder Better Faster Stronger)
* status: HTTP status (ex. 200)
* ts: Event timestamp(unix timestamp) (ex. 1538352676000)
* userAgent: User's browswer agent (ex. Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0)
* userId: User ID (ex. 30)

In [6]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



Statistics of the whole dataset

In [7]:
df.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+------------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+------------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|            length| level|         location|method|   page|        registration|        sessionId|                song|            status|                  ts|           userAgent|            userId|
+-------+------------------+----------+---------+------+------------------+--------+------------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+------------------+
|  count|            228108|    286500|   278154|278154|            286500|  278154|            228108|286500|           278154|286500| 286500|     

Statistics of the `artist` column

In [8]:
df.describe('artist').show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161377|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



Statistics of the `sessionId` column

In [9]:
df.describe('sessionId').show()

+-------+-----------------+
|summary|        sessionId|
+-------+-----------------+
|  count|           286500|
|   mean|1041.526554973822|
| stddev|726.7762634630807|
|    min|                1|
|    max|             2474|
+-------+-----------------+



Statistics of the `userId` column

In [10]:
df.describe('userId').show()

+-------+------------------+
|summary|            userId|
+-------+------------------+
|  count|            286500|
|   mean| 59682.02278593872|
| stddev|109091.94999910559|
|    min|                  |
|    max|                99|
+-------+------------------+



Total rows: 286,500

In [11]:
df.count()

286500

All the `page` events

- About: 
- Add Friend: 
- Add to Playlist: 
- Cancel: 
- Cancellation Confirmation: **TODO:**
- Downgrade: 
- Error: 
- Help: 
- Home: 
- Login: 
- Logout: 
- NextSong: 
- Register: 
- Roll Advert: 
- Save Settings: 
- Settings: 
- Submit Downgrade: 
- Submit Registration: 
- Submit Upgrade: 
- Thumbs Down: 
- Thumbs Up: 
- Upgrade: 

In [12]:
df.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



All the event logs of `userId: 99`

In [13]:
df.select(["userId", "firstname", "page", "song"]).where(df.userId == "90").collect()

[Row(userId='90', firstname='Jordan', page='Help', song=None),
 Row(userId='90', firstname='Jordan', page='NextSong', song="I'm So Sick"),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Black Sunshine'),
 Row(userId='90', firstname='Jordan', page='NextSong', song='You Could Be Happy'),
 Row(userId='90', firstname='Jordan', page='Logout', song=None),
 Row(userId='90', firstname='Jordan', page='Home', song=None),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Malana'),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Too Little_ Too Late'),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Fy Fan'),
 Row(userId='90', firstname='Jordan', page='NextSong', song="I'm Yours (Album Version)"),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Matricide'),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Invocation: Attica Blues'),
 Row(userId='90', firstname='Jordan', page='NextSong', song='Sehr kosmisch'),
 Row(userId='

## Clean data

If the below Ids are null, delete those rows:

* userId
* sessionId

In [14]:
df_valid = df.dropna(how = "any", subset = ["userId", "sessionId"])
df_valid = df_valid.filter(df_valid["userId"] != "") # `userId` can be empty string

In [15]:
print("df:         {}".format(df.count()))
print("df_valid: {}".format(df_valid.count())) # no missing values

if df.count() == df_valid.count():
    print("No missing values with userId and sessionId")
else:
    print("{} rows have been removed.".format(df.count() - df_valid.count()))

df:         286500
df_valid: 278154
8346 rows have been removed.


churn: `Cancellation Confirmation`

In [16]:
df_valid.filter("page = 'Cancellation Confirmation'").show(10)

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|                page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Cancelled|   Adriel|     M|          104| Mendoza|  null| paid|  Kansas City, MO-KS|   GET|Cancellation Conf...|1535623466000|      514|null|   200|1538943990000|"Mozilla/5.0 (Mac...|    18|
|  null|Cancelled|    Diego|     M|           56|   Mckee|  null| paid|Phoenix-Mesa-Scot...|   GET|Cancellation Conf...|1537167593000|      540|null|   200|1539033046000|"Mozilla/5.0 (iPh...|    32|
|  nu

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [46]:
churned_userIds = [x.userId for x in df_valid.filter("page = 'Cancellation Confirmation'").groupby("userId").count().select("userId").sort("userId").collect()]
churned_userIds

['100001',
 '100003',
 '100005',
 '100006',
 '100007',
 '100009',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100017',
 '100019',
 '100021',
 '100022',
 '100023',
 '100024',
 '100025',
 '101',
 '103',
 '105',
 '106',
 '12',
 '121',
 '122',
 '125',
 '129',
 '143',
 '17',
 '18',
 '200001',
 '200011',
 '200015',
 '200016',
 '200017',
 '200018',
 '200020',
 '200021',
 '200024',
 '28',
 '29',
 '3',
 '300001',
 '300007',
 '32',
 '51',
 '53',
 '54',
 '58',
 '70',
 '73',
 '87']

In [65]:
df_valid.groupby('userId').count().filter(col("userId").isin(churned_userIds)).sort("count").show()

+------+-----+
|userId|count|
+------+-----+
|   125|   11|
|100011|   23|
|100024|   36|
|100006|   44|
|   122|   62|
|100017|   75|
|100003|   78|
|100019|   89|
|    32|  108|
|300007|  143|
|   143|  149|
|200001|  158|
|100001|  187|
|100005|  216|
|     3|  254|
|200016|  268|
|100014|  310|
|100021|  319|
|200015|  349|
|   129|  412|
+------+-----+
only showing top 20 rows



In [26]:
flag_churn_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [27]:
df_valid = df_valid.withColumn("churn", flag_churn_event("page"))

In [77]:
df_valid.select(["userId", "firstname", "ts", "page", "level", "churn"]).where(df_valid.userId == "125").sort("ts").collect()

[Row(userId='125', firstname='Mason', ts=1539317144000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539317481000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539317711000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539317914000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318124000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318515000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318728000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318906000, page='NextSong', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318917000, page='Roll Advert', level='free', churn=0),
 Row(userId='125', firstname='Mason', ts=1539318918000, page='Cancel', level='free', churn=0),
 Row(userId='125', firstname=

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [81]:
df_valid.groupby("userId").max("churn").show(5)

+------+----------+
|userId|max(churn)|
+------+----------+
|100010|         0|
|200002|         0|
|   125|         1|
|    51|         1|
|   124|         0|
+------+----------+
only showing top 5 rows



# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.